In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import os
from pathlib import Path
import seaborn as sns
from tqdm import tqdm


In [2]:
path = os.getcwd() + "/Notebook"
files = os.listdir(path) 
months = [i for i in files if str(i).isnumeric()]


In [3]:
for i in tqdm(months):
    dataframes=[]
    thisdir = os.getcwd() + "/Notebook" +"/" +str(i)
    for j in os.listdir(thisdir):
        if j.isnumeric():
            dir2 = os.listdir(thisdir+"/"+str(j))
            for k in dir2:
                if 'Reis_Bestemming_Datum' in k: 
                    file_dir = thisdir + "/" + j + "/" + k
                    df = pd.read_csv(str(file_dir),sep=';')
                    #Clean stuff
                    df = df[df.AankomstHalteNaam != 'Overig']
                    df = df[df.AankomstHalteNaam != '[[ Onbekend ]]']
                    
                    df = df.dropna()

                    df = df.sort_values(by=['Datum'])
                    dataframes.append(df)
                    
    merged = pd.concat(dataframes, ignore_index=True)
    merged.to_csv('merged' + str(i) + ".csv")
        
    

100%|██████████| 12/12 [00:04<00:00,  2.71it/s]


In [4]:
#all to one file
filelist=[]
for i in tqdm(months):
    dataframes=[]
    thisdir = os.getcwd() + "/Notebook" +"/" +str(i)
    for j in os.listdir(thisdir):
        if j.isnumeric():
            dir2 = os.listdir(thisdir+"/"+str(j))
            for k in dir2:
                if 'Reis_Bestemming_Datum' in k: 
                    file_dir = thisdir + "/" + j + "/" + k
                    df = pd.read_csv(str(file_dir),sep=';')
                        #Clean stuff
                    df = df[df.AankomstHalteNaam != 'Overig']
                    df = df[df.AankomstHalteNaam != '[[ Onbekend ]]']
                    
                    df = df.dropna()

                    dataframes.append(df)
    
    merged = pd.concat(dataframes, ignore_index=True)
    filelist.append(merged)
    
    
final_merged = pd.concat(filelist, ignore_index=True)

100%|██████████| 12/12 [00:01<00:00, 11.14it/s]


In [5]:
final_merged['Modaliteit'] = final_merged.AankomstHalteCode.str.isnumeric()
final_merged.Modaliteit = final_merged.Modaliteit.replace(to_replace=[True],value='Tram of bus')
final_merged.Modaliteit = final_merged.Modaliteit.replace(to_replace=[False],value='Metro')
final_merged.to_csv("Bestemming_Datum_merged.csv")

In [6]:
final_merged.Modaliteit.value_counts()

Tram of bus    656398
Metro           29630
Name: Modaliteit, dtype: int64

In [7]:
ModaliteitTypes = pd.DataFrame(final_merged.groupby('AankomstHalteNaam')['Modaliteit'].apply(list).apply(set)).reset_index()


In [8]:
droppedduplicates = final_merged.drop_duplicates(subset=['AankomstHalteNaam']).dropna(axis=1)

In [11]:
Final_merged_one_location = pd.merge(final_merged,droppedduplicates, on="AankomstHalteNaam",how='outer')[["Datum_y", "AankomstHalteCode_x", "AankomstLat_y","AankomstHalteNaam" ,"AankomstLon_y", "AantalReizen_x", "Modaliteit_x"]]

In [12]:
Final_merged_one_location.Modaliteit_x.value_counts()

Tram of bus    656398
Metro           29630
Name: Modaliteit_x, dtype: int64

In [13]:
Final_merged_one_location.to_csv("Bestemming_Datum_merged_one_location.csv")

In [18]:
#Uur to one file 
filelist=[]
for i in tqdm(months):
    dataframes=[]
    thisdir = os.getcwd() + "/Notebook" +"/" +str(i)
    for j in os.listdir(thisdir):
        if j.isnumeric():
            dir2 = os.listdir(thisdir+"/"+str(j))
            for k in dir2:
                if 'Reis_Bestemming_Uur' in k: 
                    file_dir = thisdir + "/" + j + "/" + k
                    df = pd.read_csv(str(file_dir),sep=None,engine='python')
                        #Clean stuff
                    if "AankomstHalteNaam" in df.columns:
                        df = df[df.AankomstHalteNaam != 'Overig']
                        df = df[df.AankomstHalteNaam != '[[ Onbekend ]]']
                    
                    df = df.dropna()

                    dataframes.append(df)
    
    merged = pd.concat(dataframes, ignore_index=True)
    filelist.append(merged)
    
    
final_merged_Uur = pd.concat(filelist, ignore_index=True)



100%|██████████| 12/12 [01:07<00:00,  5.66s/it]


In [19]:
final_merged_Uur['Modaliteit'] = final_merged_Uur.AankomstHalteCode.str.isnumeric()
final_merged_Uur.Modaliteit = final_merged_Uur.Modaliteit.replace(to_replace=[True],value='Tram of bus')
final_merged_Uur.Modaliteit = final_merged_Uur.Modaliteit.replace(to_replace=[False],value='Metro')

final_merged_Uur.Datum =pd.to_datetime(final_merged_Uur.Datum)

final_merged_Uur = final_merged_Uur.sort_values(by=['Datum'])
final_merged_Uur = final_merged_Uur.drop("Unnamed: 0",axis=1)
final_merged_Uur = final_merged_Uur.dropna()
#final_merged_Uur.to_csv("Bestemming_Uur_merged.csv")

In [32]:
final_merged_Uur.head()

,Datum,UurgroepOmschrijving (van aankomst),AankomstHalteCode,AankomstHalteNaam,AankomstLat,AankomstLon,AantalReizen,Modaliteit
236050,2019-01-01,22:00 - 22:59,06045,1e Con. Huygensstraat,4.873992,52.362426,10,Tram of bus
236051,2019-01-01,22:00 - 22:59,06048,J.P. Heijestraat,4.865854,52.360377,24,Tram of bus
236052,2019-01-01,22:00 - 22:59,06062,Leidseplein,4.883246,52.363274,51,Tram of bus
236053,2019-01-01,22:00 - 22:59,06063,Leidseplein,4.880645,52.364764,41,Tram of bus
236054,2019-01-01,22:00 - 22:59,06067,Leidseplein,4.883014,52.364316,32,Tram of bus


In [22]:
droppedduplicates = final_merged_Uur.drop_duplicates(subset=['AankomstHalteNaam']).dropna(axis=1)

In [23]:
Final_merged_Uur_one_location = pd.merge(final_merged_Uur,droppedduplicates, on="AankomstHalteNaam",how='outer')[["Datum_y", "AankomstHalteCode_x", "AankomstLat_y","AankomstHalteNaam" ,"AankomstLon_y", "AantalReizen_x", "Modaliteit_x"]]

In [31]:
Final_merged_Uur_one_location.columns = ["Datum", "AankomstHalteCode", "AankomstLat","AankomstHalteNaam" ,"AankomstLon", "AantalReizen", "Modaliteit"]

In [33]:
Final_merged_Uur_one_location.to_csv("Bestemming_Uur_merged_one_location.csv")

In [8]:
pd.set_option('display.max_columns', 1000)  # or 1000
pd.set_option('display.max_rows', 1000)  # or 1000
pd.set_option('display.max_colwidth', 199)  # or 199

In [33]:
final_merged_Uur.to_csv("Bestemming_Uur_merged.csv")

In [34]:
final_merged_Uur

,Datum,UurgroepOmschrijving (van aankomst),AankomstHalteCode,AankomstHalteNaam,AankomstLat,AankomstLon,AantalReizen,Modaliteit
232600,2019-01-01,11:00 - 11:59,08239,Rietlandpark,4.934136,52.372962,14,Tram of bus
232599,2019-01-01,11:00 - 11:59,08095,Alexanderplein,4.919699,52.363415,12,Tram of bus
232598,2019-01-01,11:00 - 11:59,08080,Spiegelgracht,4.886912,52.361070,86,Tram of bus
232597,2019-01-01,11:00 - 11:59,08078,Vijzelgracht,4.890934,52.359846,12,Tram of bus
232604,2019-01-01,11:00 - 11:59,08252,IJburg,5.004326,52.350956,17,Tram of bus
...,...,...,...,...,...,...,...,...
9334753,2020-12-13,23:00 - 23:59,WLP,Waterlooplein,4.902982,52.366979,14,Metro
9333543,2020-12-13,15:00 - 15:59,06030,Rozengracht,4.875909,52.371880,22,Tram of bus
9334739,2020-12-13,23:00 - 23:59,ND,Noord,4.931232,52.401172,140,Metro
9334738,2020-12-13,23:00 - 23:59,KEN,Kraaienneststation,4.979635,52.316325,39,Metro
